# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


# Part II. Apache Cassandra

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of the denormalized data in the <font color=red>**event_datafile_new.csv**</font> after running the following cells:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# Keyspace creation
query = """
    CREATE KEYSPACE IF NOT EXISTS cassandra_sparkify
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1 } 
"""
session.execute(query)

#### Set Keyspace

In [9]:
session.set_keyspace('cassandra_sparkify')

## Table 1 session songs

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Table 1 creation

In [10]:
# Based on query1, the partition keys are sessionId and itemInSession as they are mainly used to filter the records
query = """
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY (sessionID, itemInSession)) 
"""
session.execute(query)

### Table 1 data insertion

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
data = session.execute("""SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4""")

for record in data:
    print(record.artist)
    print(record.song_title)
    print(record.song_length)
    print('*')

Faithless
Music Matters (Mark Knight Dub)
495.30731201171875
*


## Table 2 user songs

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Table 2 creation

In [13]:
# Based on query2 userId and sessionId are used as the partition keys as they are used for filtering the records
# Given that the results should be sorted based on itemInSession, it was used as the clustering key
query = """
    CREATE TABLE IF NOT EXISTS user_songs
    (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,
    PRIMARY KEY((userId, sessionId), itemInSession))
    WITH CLUSTERING ORDER BY (itemInSession DESC)
    """
session.execute(query)
                    

### Table 2 data insertion

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
data = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182""")

for record in data:
    print(record.iteminsession)
    print(record.artist)
    print(record.song)
    print(record.firstname)
    print(record.lastname)
    print('*')

3
Lonnie Gordon
Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
Sylvie
Cruz
*
2
Sebastien Tellier
Kilometer
Sylvie
Cruz
*
1
Three Drives
Greece 2000
Sylvie
Cruz
*
0
Down To The Bone
Keep On Keepin' On
Sylvie
Cruz
*


## Table 3 application data

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Table 3 creation

In [16]:
# Based on query3, song should be used as the partition key as the records are filtered based on it
#Given that the song column is not unique, we had to use another unique partition key which is userId
query = """
    CREATE TABLE IF NOT EXISTS app_data
    (song text, userId int, firstName text, lastName text,
    PRIMARY KEY(song, userId))
    """
session.execute(query)            
                    

### Table 3 data insertion

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_data (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
data = session.execute("""SELECT firstName, lastName FROM app_data WHERE song = 'All Hands Against His Own'""")

for record in data:
    print(record.firstname)
    print(record.lastname)
    print('*')

Jacqueline
Lynch
*
Tegan
Levine
*
Sara
Johnson
*


### Drop the tables before closing out the sessions

In [19]:
session.execute("""DROP TABLE session_songs""")
session.execute("""DROP TABLE user_songs""")
session.execute("""DROP TABLE app_data""")

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()